In [1]:
import requests

In [2]:
response=requests.get("https://www.balldontlie.io/api/v1/games")

In [3]:
response.json()["data"]

[{'id': 47179,
  'date': '2019-01-30T00:00:00.000Z',
  'home_team': {'id': 2,
   'abbreviation': 'BOS',
   'city': 'Boston',
   'conference': 'East',
   'division': 'Atlantic',
   'full_name': 'Boston Celtics',
   'name': 'Celtics'},
  'home_team_score': 126,
  'period': 4,
  'postseason': False,
  'season': 2018,
  'status': 'Final',
  'time': ' ',
  'visitor_team': {'id': 4,
   'abbreviation': 'CHA',
   'city': 'Charlotte',
   'conference': 'East',
   'division': 'Southeast',
   'full_name': 'Charlotte Hornets',
   'name': 'Hornets'},
  'visitor_team_score': 94},
 {'id': 48751,
  'date': '2019-02-09T00:00:00.000Z',
  'home_team': {'id': 2,
   'abbreviation': 'BOS',
   'city': 'Boston',
   'conference': 'East',
   'division': 'Atlantic',
   'full_name': 'Boston Celtics',
   'name': 'Celtics'},
  'home_team_score': 112,
  'period': 4,
  'postseason': False,
  'season': 2018,
  'status': 'Final',
  'time': '     ',
  'visitor_team': {'id': 13,
   'abbreviation': 'LAC',
   'city': 'LA',


Objectives: 
1. Get the probability of win of a matches and compare againist betting company
2. After finished the first objectives, use live api to extract statistic and scarp odds from betting campany


To Do:
1. Get a small portion of data of team statistic ✅
2. Pipeline it ✅
3. Build a deep learning classification
    3.1 train/test?
4. Get the probability

Optional task:
1. download data directly from kaggle
2. use make_pipeline instead

In [4]:
import numpy as np

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import make_scorer

In [6]:
import pandas as pd
df=pd.read_csv("games.csv")
df.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,12/03/2022,22101005,Final,1610612748,1610612750,2021,1610612748,104.0,0.398,0.760,...,23.0,53.0,1610612750,113.0,0.422,0.875,0.357,21.0,46.0,0
1,12/03/2022,22101006,Final,1610612741,1610612739,2021,1610612741,101.0,0.443,0.933,...,20.0,46.0,1610612739,91.0,0.419,0.824,0.208,19.0,40.0,1
2,12/03/2022,22101007,Final,1610612759,1610612754,2021,1610612759,108.0,0.412,0.813,...,28.0,52.0,1610612754,119.0,0.489,1.000,0.389,23.0,47.0,0
3,12/03/2022,22101008,Final,1610612744,1610612749,2021,1610612744,122.0,0.484,0.933,...,33.0,55.0,1610612749,109.0,0.413,0.696,0.386,27.0,39.0,1
4,12/03/2022,22101009,Final,1610612743,1610612761,2021,1610612743,115.0,0.551,0.750,...,32.0,39.0,1610612761,127.0,0.471,0.760,0.387,28.0,50.0,0


1. drop NA, unneccesary columns ✅
2. check which columns to pipeline ✅
3. get the team name instead of only id ✅

In [7]:
df.dropna(axis=0,inplace=True)
df.drop(["TEAM_ID_home","TEAM_ID_away"],axis=1,inplace=True)
team=pd.read_csv("teams.csv")
def get_team(num):
    return team[team["TEAM_ID"]==num]["NICKNAME"].values[0]
df["HOME_TEAM_NAME"]=np.vectorize(get_team)(df["HOME_TEAM_ID"])
df["HOME_TEAM_NAME"]=np.vectorize(get_team)(df["HOME_TEAM_ID"])
X_train=df.drop("HOME_TEAM_WINS",axis=1)
y_train=df[["HOME_TEAM_WINS"]]

Try to use make_pipeline below

In [8]:
"""
preproc_numerical_baseline = make_pipeline(StandardScaler())
preproc_categorical_baseline = make_pipeline(OneHotEncoder(handle_unknown="ignore"))
preproc_baseline = make_column_transformer(
    (preproc_numerical_baseline, ['PTS_home','FG_PCT_home',
       'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home',
       'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away',
       'REB_away']),
    (preproc_categorical_baseline, ["HOME_TEAM_ID","VISITOR_TEAM_ID"]),
    remainder="drop")
preproc_baseline
"""

'\npreproc_numerical_baseline = make_pipeline(StandardScaler())\npreproc_categorical_baseline = make_pipeline(OneHotEncoder(handle_unknown="ignore"))\npreproc_baseline = make_column_transformer(\n    (preproc_numerical_baseline, [\'PTS_home\',\'FG_PCT_home\',\n       \'FT_PCT_home\', \'FG3_PCT_home\', \'AST_home\', \'REB_home\',\n       \'PTS_away\', \'FG_PCT_away\', \'FT_PCT_away\', \'FG3_PCT_away\', \'AST_away\',\n       \'REB_away\']),\n    (preproc_categorical_baseline, ["HOME_TEAM_ID","VISITOR_TEAM_ID"]),\n    remainder="drop")\npreproc_baseline\n'

In [9]:
num_transformer = Pipeline([('scaler', StandardScaler())])
cat_transformer = OneHotEncoder(handle_unknown='ignore',sparse=False)
preprocessor = ColumnTransformer([
    ('num_tr', num_transformer, ['PTS_home','FG_PCT_home',
       'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home',
       'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away',
       'REB_away']),
    ('cat_tr', cat_transformer, ["HOME_TEAM_ID","VISITOR_TEAM_ID"])],remainder='passthrough')

In [10]:
X_train_transformed = preprocessor.fit_transform(X_train)

In [11]:
display(pd.DataFrame(X_train_transformed).head(3))

,0,1,2,3,4,5,6,7,8,9,...,67,68,69,70,71,72,73,74,75,76
0,0.067855,-1.100399,0.002931,-0.204546,0.05084,1.457963,0.952268,-0.491027,1.130565,0.068852,...,0.0,0.0,0.0,0.0,0.0,12/03/2022,22101005,Final,2021,Heat
1,-0.159858,-0.305731,1.721076,0.653073,-0.528594,0.400834,-0.696567,-0.545055,0.637409,-1.283337,...,0.0,0.0,0.0,0.0,0.0,12/03/2022,22101006,Final,2021,Bulls
2,0.371473,-0.853169,0.529299,-0.284948,1.016563,1.306945,1.40195,0.715595,2.33928,0.359255,...,0.0,0.0,0.0,0.0,0.0,12/03/2022,22101007,Final,2021,Spurs


In [12]:
X_train1=pd.DataFrame(X_train_transformed)

HOME_TEAM_WINS    int64
dtype: object

In [15]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import Sequential, layers


2022-03-22 22:27:13.879138: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-22 22:27:13.879174: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [27]:
X_train_transformed.shape[-1]

77

In [17]:
model =Sequential()
model.add(layers.Dense(10,activation="relu",input_dim=77))
model.add(layers.Dense(7, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

2022-03-22 22:27:14.689555: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: UNKNOWN ERROR (100)
2022-03-22 22:27:14.689601: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-OQ3QMKM): /proc/driver/nvidia/version does not exist
2022-03-22 22:27:14.690023: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [31]:
model.fit(X_train1, y_train, batch_size=32, epochs=500)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [36]:
df=pd.read_csv("games.csv")
df.dtypes

GAME_DATE_EST        object
GAME_ID               int64
GAME_STATUS_TEXT     object
HOME_TEAM_ID          int64
VISITOR_TEAM_ID       int64
SEASON                int64
TEAM_ID_home          int64
PTS_home            float64
FG_PCT_home         float64
FT_PCT_home         float64
FG3_PCT_home        float64
AST_home            float64
REB_home            float64
TEAM_ID_away          int64
PTS_away            float64
FG_PCT_away         float64
FT_PCT_away         float64
FG3_PCT_away        float64
AST_away            float64
REB_away            float64
HOME_TEAM_WINS        int64
dtype: object

In [41]:
df1=df.drop(["GAME_DATE_EST","GAME_ID","GAME_STATUS_TEXT","HOME_TEAM_ID","VISITOR_TEAM_ID","SEASON","TEAM_ID_home","TEAM_ID_away"],axis=1)

In [42]:
x_train=df1.drop("HOME_TEAM_WINS",axis=1)
y_train=df1[["HOME_TEAM_WINS"]]

In [ ]:
model =Sequential()
model.add(layers.Dense(10,activation="relu",input_dim=len(X)))
model.add(layers.Dense(7, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))